In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed

seed(1)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor 

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
df=pd.read_excel('Pre-Processed-Data.xlsx')
df.head()

,Dew Point,Wind Speed,Pressure,Pressure Grad
0,4.8,2.47,1029.0,-8
1,8.2,7.42,1021.4,0
2,6.8,6.81,1021.8,11
3,3.6,3.94,1033.7,-1
4,6.1,3.33,1033.4,-11


In [3]:
X = df.drop(['Wind Speed'], axis=1)
#Assign the Target column as the output 
Y= df['Wind Speed']

In [4]:
X_norm=(X-X.min())/(X.max()-X.min())
X_norm

,Dew Point,Pressure,Pressure Grad
0,0.487805,0.752055,0.449275
1,0.606272,0.647945,0.565217
2,0.557491,0.653425,0.724638
3,0.445993,0.816438,0.550725
4,0.533101,0.812329,0.405797
...,...,...,...
1091,0.411150,0.706849,0.623188
1092,0.470383,0.767123,0.594203
1093,0.595819,0.795890,0.594203
1094,0.564460,0.831507,0.565217


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_norm, Y, test_size=0.2, random_state=0)

In [8]:
model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation="sigmoid", kernel_initializer='normal'))
model.add(Dropout(0.2)) #dropping a few neurons for generalizing the model     
model.add(Dense(1, activation="linear", kernel_initializer='normal'))
adam = Adam(learning_rate=1e-3, decay=1e-3)

# Compile model
model.compile(loss="mean_squared_error", optimizer='adam', metrics=['mse','mae'])

In [9]:
print('Fit model...')
filepath="/home/m-marouni/Documents/CE-901/Heathrow/best_weights"
checkpoint = ModelCheckpoint(filepath, monitor='val_mae', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_mae', patience=100, verbose=1, mode='min')
callbacks_list = [checkpoint, early_stopping]

log = model.fit(x_train, y_train,
          validation_split=0.40, batch_size=30, epochs=1000, shuffle=True, callbacks=callbacks_list)

Fit model...
Epoch 1/1000
18/18 [==============================] - 12s 36ms/step - loss: 43.6293 - mse: 43.6293 - mae: 6.0681 - val_loss: 36.4766 - val_mse: 36.4766 - val_mae: 5.5491

Epoch 00001: val_mae improved from inf to 5.54912, saving model to /home/m-marouni/Documents/CE-901/Heathrow/best_weights
INFO:tensorflow:Assets written to: /home/m-marouni/Documents/CE-901/Heathrow/best_weights/assets
Epoch 2/1000
18/18 [==============================] - 0s 5ms/step - loss: 39.2407 - mse: 39.2407 - mae: 5.6907 - val_loss: 33.0844 - val_mse: 33.0844 - val_mae: 5.2344

Epoch 00002: val_mae improved from 5.54912 to 5.23439, saving model to /home/m-marouni/Documents/CE-901/Heathrow/best_weights
INFO:tensorflow:Assets written to: /home/m-marouni/Documents/CE-901/Heathrow/best_weights/assets
Epoch 3/1000
18/18 [==============================] - 0s 3ms/step - loss: 36.5446 - mse: 36.5446 - mae: 5.4144 - val_loss: 29.8729 - val_mse: 29.8729 - val_mae: 4.9178

Epoch 00003: val_mae improved from 5

In [17]:
df_test=pd.read_excel('London-Test-Data.xlsx')
df_test.head()

,Wind Speed,Pressure,Pressure Gradient,Dew Point
0,3,1017.1,-2,10.5
1,3,1015.3,-4,12.7
2,5,1011.5,-5,14.7
3,6,1007.3,-4,13.9
4,7,1004.2,-5,11.6


In [18]:
X2 = df_test.drop(['Wind Speed'], axis=1)
#Assign the Target column as the output 
Y2 = df_test['Wind Speed']

In [19]:
X2_norm=(X2-X2.min())/(X2.max()-X2.min())
X2_norm

,Pressure,Pressure Gradient,Dew Point
0,0.625442,0.1875,0.000000
1,0.561837,0.0625,0.523810
2,0.427562,0.0000,1.000000
3,0.279152,0.0625,0.809524
4,0.169611,0.0000,0.261905
5,0.000000,1.0000,0.380952
6,0.420495,0.7500,0.428571
7,0.689046,0.3750,0.619048
8,0.724382,0.0625,0.666667
9,0.586572,0.1875,0.238095


In [20]:
new_predictions=model.predict(X2_norm)

In [21]:
new_predictions

array([[7.32432  ],
       [8.770636 ],
       [9.764495 ],
       [9.245988 ],
       [8.95782  ],
       [3.8484216],
       [4.849235 ],
       [7.000834 ],
       [8.886296 ],
       [7.6522794],
       [8.780792 ],
       [6.823234 ],
       [6.806063 ],
       [6.13311  ],
       [5.926943 ],
       [6.5306535],
       [6.6635823]], dtype=float32)

In [22]:
mean_absolute_error(Y2, new_predictions)

3.3864346672506893

In [23]:
mean_squared_error(Y2, new_predictions)

13.893739542299954

In [24]:
df_test['Predictions'] = new_predictions
#Save the dataframe as csv file
df_test.to_csv('London_results.xlsx', index = False)